In [1]:
!pip install mutagen

/home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/.venv/bin/pip: line 2: /home/mox/Documents/coding_projects/personal projects/ipod_recover_rename/.venv/bin/python: No such file or directory


In [19]:
import pathlib
import os
import mutagen as mg
from mutagen.id3 import ID3, TIT2

import itertools
import shutil


In [3]:
cwd = pathlib.Path.cwd()

if cwd.name == 'ipod_recover_rename':
    print("Path is project root")
else:
    print('Current Working Directory is:', cwd)
    print("Recomend navigating to the project root in terminal and opening your chosen idea via the command 'code .'")
    raise NameError('Please correct current working directory to the project root')


Path is project root


In [4]:
old_path = cwd / "Music"
after_path = cwd / "After"

In [5]:
if not os.path.exists(after_path): 
    # if the After directory is not present  
    # then create it. 
    os.makedirs(after_path) 

In [6]:
f_list = os.listdir(old_path)
# copies music to after folder for processing of a copy of the music, ensuring no loss
for file in f_list:
    # prevents copying of the hidden files associated with ituns and ipods
    if file.startswith('._'):
        pass
    else:
        shutil.copytree(old_path / file, after_path / file)


In [7]:
# https://stackoverflow.com/questions/17547273/flatten-complex-directory-structure-in-python
#Move all files from music to (created) intermediary
def move(destination):
    all_files = []
    first_loop_pass = True
    for root, _dirs, files in os.walk(destination):
        if first_loop_pass:
            first_loop_pass = False
            continue
        for filename in files:
            all_files.append(os.path.join(root, filename))
    for filename in all_files:
        shutil.move(filename, destination)

In [8]:
move(after_path)

In [9]:
walk = list(os.walk(after_path))
for path, _, _ in walk[::]:
    if len(os.listdir(path)) == 0:
        # remove empty directories
        os.removedirs(path)


In [ ]:
test_mp3 = after_path / 'CUYL.mp3'
test_m4a = after_path / 'XWHW.m4a'
test_m4p = after_path / 'DMZZ.m4p'
test_flac = after_path / 'ASCS.flac'
test_wav = after_path / 'GSDE.wav'
test_wma = after_path / 'AFES.wma'

test_files = [test_mp3, test_m4a, test_m4p, test_flac, test_wav, test_wma]

In [ ]:
# List all file types for appropriate parsing
compatible_types = {
    'id3': ('.mp3', '.wav'),
    'other': ('.m4a', '.m4p', '.wma', '.flac')
    #TODO: add support for other using mugagen tag readers schemes
}

def get_rename_dict(file):
    file_ext = file.suffix
    #handle ID3 compatible filetypes
    if file_ext in compatible_types['id3']:
        file_path = after_path / file
        audio_file = mg.File(file_path)
        return {'artist':audio_file['TPE1'].text[0],
                'album':audio_file['TALB'].text[0],
                'title':audio_file['TIT2'].text[0]}
    #TODO add suport for others in compatible types dict
    

In [ ]:
for file in test_files:
    try:
        name_dict = get_rename_dict(file) 
        new_name_string = f'{name_dict['artist']}_{name_dict['album']}_{}'
    except:
        print(file.name, "did not work", file.suffix, "is unsupported")

{'artist': 'Devin the Dude', 'album': "Just Tryin' Ta Live", 'title': "It's A Shame (feat. Poo Bear)"}
None
None
ASCS.flac did not work .flac is unsupported
{'artist': 'Masatoshi Yanagi', 'album': 'Hi-Fi RUSH B-Sides', 'title': 'Arcade! Challenge!'}
None


TODO: 
    1. rename files to artist_album_title.ext
    2. Add files to artist-album-song structure
       1. walk file list and make [artist, album, title.ext]
       2. check if artist-album folders exist, make them if DNE 
       3. rename to title.ext and move to appropriate folder tree

In [40]:
get_rename_dict(test_mp3)

{'artist': ['The Devil Makes Three'],
 'album': TALB(encoding=<Encoding.LATIN1: 0>, text=['The Devil Makes Three']),
 'title': TIT2(encoding=<Encoding.LATIN1: 0>, text=['Graveyard'])}

In [77]:
file = 'tree_test.mp3'

name_dict = get_rename_dict(after_path / file) 
artist = name_dict['artist']
artist_path = after_path / artist
album = name_dict['album']
album_path = artist_path / album
title = name_dict['title']
# if artist folder DNE, make it
if not os.path.isdir(artist_path):
    os.mkdir(artist_path)
# if album folder DNE, make it
if not os.path.isdir(album_path):
    os.mkdir(album_path)

shutil.copy(after_path / 'tree_test.mp3', after_path / artist / album / title)


PosixPath("/home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/After/Devin the Dude/Just Tryin' Ta Live/It's A Shame (feat. Poo Bear)")